# **This notebook is for doing imputations on the DataFrames null values.** 

#### We are going to USE ML to fill the dataframes missing NAN values. We are beggining with The values that have the least NAN values and build up the prediction strength from ground up.

In [52]:
import pandas as pd
import numpy as np

Read in the repaird taxi trip csv:

In [53]:
df = pd.read_csv("../../data/taxi_trip_pricing_numeric_repaired.csv")

Identify the "Per_Minute_Rate" column. And store the rows with nana values away.

In [54]:
# Store the rows with NaN values in "Per_Minute_Rate" column in df and remove them from df
nan_rows = df[df["Per_Minute_Rate"].isna()] # We will use these rows later to predict the missing values
df_ml = df[df["Per_Minute_Rate"].notna()]

**df_ml** är dataframen med 995 rader nu men vi ska köra one hot encodning på sträng columnerna också **OCH** tar bort alla rader som innehåller minst ett NAN

In [55]:
# perform one hot encoding on categorical columns
df_ml = pd.get_dummies(df_ml, columns=["Time_of_Day", "Day_of_Week", "Traffic_Conditions", "Weather"], drop_first=True)

# Remove nan values from df_ml
df_ml = df_ml.dropna()


In [56]:
df_ml.isna().sum()  # Check if there are any NaN values left

Trip_Distance_km             0
Passenger_Count              0
Base_Fare                    0
Per_Km_Rate                  0
Per_Minute_Rate              0
Trip_Duration_Minutes        0
Trip_Price                   0
Time_of_Day_Evening          0
Time_of_Day_Morning          0
Time_of_Day_Night            0
Day_of_Week_Weekend          0
Traffic_Conditions_Low       0
Traffic_Conditions_Medium    0
Weather_Rain                 0
Weather_Snow                 0
dtype: int64

Now we are going to split the data into train test split

In [ ]:
from sklearn.model_selection import train_test_split

X,y = df_ml.drop(columns=["Per_Minute_Rate"]), df_ml["Per_Minute_Rate"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# LINNEAR REGRESSION MODEL TEST FOR **PER_MINUTE_RATE**

In [61]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

scaler = MinMaxScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
 
ln_model = LinearRegression()
ln_model.fit(X_train_scaled, y_train)

y_pred = ln_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R^2 Score: {r2:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")


Mean Squared Error: 0.01
Root Mean Squared Error: 0.10
R^2 Score: 0.23
Mean Absolute Error: 0.08


# Random forest MODEL TEST FOR **PER_MINUTE_RATE**

In [62]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"Random Forest - Mean Squared Error: {mse_rf:.2f}")
print(f"Random Forest - Root Mean Squared Error: {rmse_rf:.2f}")
print(f"Random Forest - R^2 Score: {r2_rf:.2f}")
print(f"Random Forest - Mean Absolute Error: {mae_rf:.2f}")


Random Forest - Mean Squared Error: 0.01
Random Forest - Root Mean Squared Error: 0.09
Random Forest - R^2 Score: 0.38
Random Forest - Mean Absolute Error: 0.07


## RANDOM FOREST ÄR VINNAREN. Vi tillämpar denna på. Tränar upp den maximalt och sen kör prediktion på nan värdena

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Day_of_Week_Weekend,Traffic_Conditions_Low,Traffic_Conditions_Medium,Weather_Rain,Weather_Snow
0,19.35000,3.0,3.56,0.80,0.320000,53.82,36.262400,False,True,0,0,1,False,False,0
1,36.87000,1.0,2.70,1.21,0.150000,37.27,52.903200,True,False,0,1,0,False,False,0
2,30.33000,4.0,3.48,0.51,0.150000,116.81,36.469800,True,False,0,0,1,False,False,0
3,8.64000,3.0,2.93,0.63,0.320000,22.64,15.618000,True,False,0,0,0,False,False,0
4,8.64000,2.0,2.55,1.71,0.480000,89.33,60.202800,False,False,0,1,0,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,100.38042,3.0,4.46,NaN,0.633896,89.21,329.913004,False,False,0,0,0,True,True,0
920,34.55000,1.0,2.47,1.03,0.373900,NaN,63.615000,False,False,0,0,0,False,True,0
921,17.41000,NaN,2.05,1.86,0.339300,39.98,NaN,False,True,0,0,0,True,False,0
922,14.34000,1.0,3.69,1.85,0.229600,NaN,38.035000,True,False,0,0,0,False,True,0
